In [38]:
import numpy as np
impor math
import random
from scipy.stats import norm
random.seed(10)

In [53]:
def d1(S,K,T,r,sigma):
    return(math.log(S/K)+(r+sigma**2/2.)*T)/(sigma*math.sqrt(T))
def d2(S,K,T,r,sigma):
    return d1(S,K,T,r,sigma)-sigma*math.sqrt(T)
def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*math.exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))

In [55]:
bs_call(10, 12, 1, 0.23, 0.4)

1.7919270785071006

<h2> Stratification </h2>
<ol>
    <li>Partition (-inf, inf) into B bins </li>
    <li> Set $V_{i} = \frac{1}{B}(U_{i} + (i-1))$ , i = 0...B and generate desired number of random samples $N_{B}$ of V in the i th bin </li>
    <li>Apply $\Phi^{-1}(V_{i})$ to get the desired normal random number from the bin and calculate $C_{i}$  from each bin</li>
    <li> Average the $\overline{C}_{i}$ over the number of bins to get overall estimate $\overline{C}$</li>
    <li> Calculate the standard error in previous case </li>
</ol>

In [51]:
n = 10000 #total sample size
B = 100 #number of bins
NB = n//B #number of sample per bucket

S0, K, mu, sigma = 10, 12, 0.23, 0.4
nu = mu-sigma**2/2
T = 1

ST = np.zeros(NB) #generate 
Ci = np.zeros(NB)
Cbar = 0
varr = 0 
for i in np.arange(0, B):#loop from 0 to B-1
    u = np.random.uniform(size=NB)
    z = norm.ppf((u+i)/B)
    for j in np.arange(0, NB) :
        ST[j] = S0*math.exp(nu*T+sigma*math.sqrt(T)*z[j])
        Ci[j] = math.exp(-mu*T) *max(ST[j]-K, 0)
    Cbar = Cbar + np.average(Ci)
    varr = varr + np.var(Ci)

C = Cbar/B
SE = math.sqrt(varr/NB)/B

In [52]:
print(C, SE)

1.793180141695758 0.0040870948814573


Reduce waste when S < K for call Option

$S_{T} = S_{0} * e^{[r-\sigma^{2}]}$ 

$S_{T} < K$ 

In [60]:
n = 10000 #total sample size
B = 100 #number of bins
NB = n//B #number of sample per bucket

S0, K, mu, sigma = 10, 12, 0.23, 0.4
nu = mu-sigma**2/2
T = 1

ST = np.zeros(NB) #generate 
Ci = np.zeros(NB)
Cbar = 0
varr = 0 
L = (math.log(K/S0)-(mu-sigma**2/2)*T)/(sigma*T)

for i in np.arange(0, B):#loop from 0 to B-1
    u = np.random.uniform(size=NB)
    v = (u+i)/B
    z = norm.ppf(v*(1-norm.cdf(L))+norm.cdf(L))
    for j in np.arange(0, NB) :
        ST[j] = S0*math.exp(nu*T+sigma*math.sqrt(T)*z[j])
        Ci[j] = math.exp(-mu*T) *max(ST[j]-K, 0)
    Cbar = Cbar + np.average(Ci)
    varr = varr + np.var(Ci)

In [59]:
print(C, SE)

1.793180141695758 0.0040870948814573


<h2> Control Variable </h2>